In [50]:
from timeit import timeit

In [58]:
op = """
import numpy as np
from scipy.optimize import minimize
from dipy.core.optimize import Optimizer
def rosen(x):
     return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
res = minimize(rosen, x0, method='nelder-mead', options={'xtol': 1e-8,})
"""

In [59]:
optm = """
import numpy as np
from scipy.optimize import minimize
from dipy.core.optimize import Optimizer
def rosen(x):
     return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
res = Optimizer(rosen, x0, method='nelder-mead', options={'xtol': 1e-8,})
"""

In [60]:
print("Time taken for core.Optimize :",timeit(optm, number=1000))
print("Time taken for scipy.optimize :",timeit(op, number=1000))

Time taken for core.Optimize : 29.677384773000085
Time taken for scipy.optimize : 29.868785957


In [55]:
dipy_Optimizer = """
from dipy.reconst.ivim import _ivim_error, ivim_function
from dipy.core.gradients import gradient_table, generate_bvecs
from dipy.core.optimize import Optimizer
from scipy.optimize import minimize
import numpy as np

bvals = np.array([0., 10., 20., 30., 40., 60., 80., 100.,
                  120., 140., 160., 180., 200., 220., 240.,
                  260., 280., 300., 350., 400., 500., 600.,
                  700., 800., 900., 1000.])
N = len(bvals)
bvecs = generate_bvecs(N)
gtab = gradient_table(bvals, bvecs.T)

params = [1.0, 0.132, 0.00885, 0.000921]
x0 = [[1., 0.5, 0.001, 0.0001],]

data = np.array([ivim_function(params, bvals),])
sum_sq = lambda params, bvals, data: np.sum(
        _ivim_error(params, bvals, data)**2)

num_voxels = data.shape[0]

result = np.empty(num_voxels, dtype=object)

for vox in range(num_voxels):
    res = Optimizer(sum_sq,
                    x0[vox],
                    args=(bvals, data[vox]))
    result[vox] = res
"""

In [56]:
scipy_optimizer = """
from dipy.reconst.ivim import _ivim_error, ivim_function
from dipy.core.gradients import gradient_table, generate_bvecs
from dipy.core.optimize import Optimizer
from scipy.optimize import minimize
import numpy as np

bvals = np.array([0., 10., 20., 30., 40., 60., 80., 100.,
                  120., 140., 160., 180., 200., 220., 240.,
                  260., 280., 300., 350., 400., 500., 600.,
                  700., 800., 900., 1000.])
N = len(bvals)
bvecs = generate_bvecs(N)
gtab = gradient_table(bvals, bvecs.T)

params = [1.0, 0.132, 0.00885, 0.000921]
x0 = [[1., 0.5, 0.001, 0.0001],]

data = np.array([ivim_function(params, bvals),])
sum_sq = lambda params, bvals, data: np.sum(
        _ivim_error(params, bvals, data)**2)

num_voxels = data.shape[0]

result = np.empty(num_voxels, dtype=object)

for vox in range(num_voxels):
    res = minimize(sum_sq,
                    x0[vox],
                    args=(bvals, data[vox]))
    result[vox] = res
"""

In [57]:
print("Time taken for core.Optimize :",timeit(dipy_Optimizer, number=10))
print("Time taken for scipy.Optimize :",timeit(scipy_optimizer, number=10))

Time taken for core.Optimize : 18.195407083000646
Time taken for scipy.Optimize : 18.237980531000176
